# Libraries / Variables / Functions

In [76]:
import time
import Levenshtein
import random
import numpy as np

data=[]                     #keep all docs
data_in_bags=[]             #keeps all words in each doc
data_in_shingles=[]         #keeps all shingles in each doc                 
jac_sim_matrix=np.zeros((491, 491))  #keeps all jac for all pairs
Universal_set=set([])  #all the k-shingles in all docs
Universal_list=[]
one_hundred_hash=[]  #save 100 hash information
one_hundred_hash_results=[]  #save 100 hash answers on 491 docs
k=3            #size of shingles
matrix_text=[] #tells us which text has which words
num_of_hash=2
num_of_bands=10
hash_functions=[]  #keeps all functions in a band
bands=[]           #keep all bands and the hash functions within them
hash_results_bands=[]    #keeps all the answers of hash functions in all bands
hash_pairs=[]      #final result from banding (all two docs with possibly more 
                   #than 0.3 jaccard similarty)

def jaccard(a,b):
    return len(a.intersection(b))/len(a.union(b))


def minHash(list,a,b,p):    #ax+b mod p daryaft mikone va listi az hameye 
                            #kalamati ke yek document dare
                            # list zir majmue az 3000 kalame madar ast 
    min=34231
    min_index=-1
    for i in range (0,len(list)):
        temp=(a*list[i]+b)%p
        if temp < min:
            min= temp
    return min


def hash_sim(list1,list2):
    count=0
    for i in range (0,len(list1)):
        if list1[i]==list2[i]:
            count+=1
    return round(count/len(list1),4)
            

def partition(hash_results):      #natayej hash 491 doc ra besurate list migirad 
                                  #va mige koduma ba ham moghayese beshan 
                                  #(tedad hash ha mitavanad harchi bashad)
    spot=[0]*491
    partitions=[]
    for i in range (0,491):
        if spot[i]==0:
            gather=[]
            spot[i]=1
            gather.append(i)
            for j in range (i+1,491):
                if hash_results[j]==hash_results[i]:
                    spot[j]=1
                    gather.append(j)
            partitions.append(gather)
    return partitions


def pair_producer(list):   #yek listi as list haro migire (natijeye partition=input) 
                           #va zoj hayi ke bayad
                           #bayad moghayese beshan vali hanuz tu hash_pairs nistand 
                           #ro vared hash_pairs mikone
        for i in range (0,len(list)):
            for j in range (0,len(list[i])):
                for k in range (j+1,len(list[i])):
                    tup=(list[i][j],list[i][k])
                    check=tup in hash_pairs
                    if check==False:
                        hash_pairs.append(tup)



# Read all the data into a list called data

In [77]:
for i in range (0,491):
    file=open('sample/'+ str(i)+'.txt', 'r', encoding="utf8")
    data.append(file.read())
    file.close()

# Calculating edit distances

In [12]:
file1=open('sample/distances.txt', 'w')
file2= open('sample/max_distances.txt', 'w')
file1.write("Pairs    Edit_Similarity\n")
file2.write("MaxPairs  Max Edit_Similarity\n")


start = time.time()
for i in range (0,491):
    max_sim=0
    max_sim_index=-1
    
    for j in range (i+1,491):
        max_string_length=max(len(data[i]),len(data[j]))
        edit_similarity=\
        (max_string_length - Levenshtein.distance(data[i],data[j]))/max_string_length
        
        if(edit_similarity>=max_sim):
            max_sim=edit_similarity
            max_sim_index=j
        
        file1.write(str(i)+" "+str(j)+"        "+str(round(edit_similarity, 4))+"\n")
        
    file2.write(str(i)+" "+str(max_sim_index)+"        "+str(round(max_sim, 4))+"\n")
        

file1.close()
file2.close()
end = time.time()

print('It took', round((end-start)/60,2), \
      'minutes to calculate edit distances and their maximum pairs')

It took 0.0 minutes to calculate edit distances and their maximum pairs


# Calculating Jaccard  (bags)

In [78]:
for j in range (0,491):
    words=data[j].split()
    data_in_bags.append(words)

    
file1=open('sample/jac_sim_bags.txt', 'w')
file2= open('sample/max_jac_sim_bags.txt', 'w')
file1.write("Pairs    Jac_Similarity_Bags\n")
file2.write("MaxPairs  Max Jac_Similarity_Bags\n")


start = time.time()
for i in range (0,491):
    max_jac_sim=0
    max_jac_sim_index=-1
    
    for j in range (i+1,491):        
        jac_similarity= jaccard(set(data_in_bags[i]),set(data_in_bags[j]))
        
        if(jac_similarity>=max_jac_sim):
            max_jac_sim=jac_similarity
            max_jac_sim_index=j
        
        file1.write(str(i)+" "+str(j)+"        "+str(round(jac_similarity, 4))+"\n")
        
    file2.write(str(i)+" "+str(max_jac_sim_index)+"        "+str(round(max_jac_sim, 4))+"\n")
        

file1.close()
file2.close()
end = time.time()

print('It took', round((end-start),2), ' seconds to calculate Jaccard distances\
 and their maximum pairs (BAG OF WORDS)')        


It took 15.56  seconds to calculate Jaccard distances and their maximum pairs (BAG OF WORDS)


# Calculating Jaccard  (Shingles)

In [79]:
for j in range (0,491):
    myset=set([])
    for i in range(1,len(data[j])-k-1):
        shingle=data[j][i:i+k]
        temp_str=''        
        for r in range(k):
            temp_str+=str(ord(shingle[r]))
        myset.add(int(temp_str))
    data_in_shingles.append(myset)
    
    
file1=open('sample/jac_sim_Shingles.txt', 'w')
file2= open('sample/max_jac_sim_Shingles.txt', 'w')
file1.write("Pairs    Jac_Similarity_Shingles\n")
file2.write("MaxPairs  Max Jac_Similarity_shingles\n")

start = time.time()
for i in range (0,491):
    max_jac_sim=0
    max_jac_sim_index=-1
    
    for j in range (i+1,491):        
        jac_similarity= jaccard(data_in_shingles[i],data_in_shingles[j])
        
        if(jac_similarity>=max_jac_sim):
            max_jac_sim=jac_similarity
            max_jac_sim_index=j
        
        file1.write(str(i)+" "+str(j)+"        "+str(round(jac_similarity, 4))+"\n")
        jac_sim_matrix[i][j]=round(jac_similarity, 4)
        
    file2.write(str(i)+" "+str(max_jac_sim_index)+"        "+str(round(max_jac_sim, 4))+"\n")
        

file1.close()
file2.close()
end = time.time()

print('It took', round((end-start),2), ' seconds to calculate\
 Jaccard distances and their maximum pairs (3-SHINGLES)')        


It took 29.06  seconds to calculate Jaccard distances and their maximum pairs (3-SHINGLES)


# 100 hash functions to make comparison easier

In [80]:
n=34231 #first prime after (34,227= size of Universal set)

start = time.time()
for j in range (0,491):
    for i in range(1,len(data[j])-k-1):
        shingle=data[j][i:i+k]
        temp_str=''         
        for r in range(k):
            temp_str+=str(ord(shingle[r]))  
        mytuple=(shingle,int(temp_str))    
        Universal_set.add(mytuple)
        
end = time.time()
print('time used to compute universal words: ', end-start)

Universal_list=list(Universal_set)


start = time.time()

for i in range (0,491):
    column=[]
    for j in range (0,34227): 
        if Universal_list[j][1] in data_in_shingles[i]: 
            column.append(j)
    matrix_text.append(column)        
            
end = time.time()

print('time used to compute membership of 3-shingles in documents ', end-start)

time used to compute universal words:  9.123588562011719
time used to compute membership of 3-shingles in documents  5.899840831756592


produce 100 random hash functions

In [81]:
count=0
while True:
    a=random.randint(1,100)
    b=random.randint(1,100)
    tup=(a,b)

    check=tup in one_hundred_hash

    if check==False:
        count+=1
        one_hundred_hash.append(tup)
    if count==100: 
        break
#print(tenhash)

one_hundred_hash_result=[]
start = time.time()
for k in range (0,491):
        temp=[]
        for j in range (0,100):
            temp.append(minHash(matrix_text[k],one_hundred_hash[j][0],one_hundred_hash[j][1],34231))
        one_hundred_hash_result.append(temp)
end = time.time()

print('time used to compute 100 hash results on 491 texts: ', end-start) 
#print(len(tenhash_result))


time used to compute 100 hash results on 491 texts:  21.58146333694458


# Estimate Jaccard Similarities using fingerprints

In [82]:
file1=open('sample/jac_sim_Shingles_hash_estimate.txt', 'w')
file2= open('sample/max_jac_sim_Shingles_hash_estimate.txt', 'w')
file1.write("Pairs    Jac_Similarity_Shingles_hash_estimate\n")
file2.write("MaxPairs  Max Jac_Similarity_shingles_hash_estimate\n")

start = time.time()
for i in range (0,491):
    max_jac_sim=0
    max_jac_sim_index=-1
    
    for j in range (i+1,491):        
        jac_similarity= hash_sim(one_hundred_hash_result[i],one_hundred_hash_result[j])
        
        if(jac_similarity>=max_jac_sim):
            max_jac_sim=jac_similarity
            max_jac_sim_index=j
        
        file1.write(str(i)+" "+str(j)+"        "+str(jac_similarity)+"\n")

        
    file2.write(str(i)+" "+str(max_jac_sim_index)+"        "+str(max_jac_sim)+"\n")
        

file1.close()
file2.close()
end = time.time()

print('It took', round((end-start),2), ' seconds to calculate\
 Jaccard distances and their maximum pairs using 100 hash_estimates')  

It took 2.04  seconds to calculate Jaccard distances and their maximum pairs using 100 hash_estimates


# MinHash and Banding Technique

Create as many Hash Functions as needed

In [83]:


hash_functions=[]
bands=[]
start = time.time()
for k in range(0,num_of_bands):
    count=0
    hash_for_this_band=[]
    while True:
        
        a=random.randint(1,100)
        b=random.randint(1,100)
        tup=(a,b)
        
        check=tup in hash_functions
        
        if check==False:
            count+=1
            hash_functions.append(tup)
            hash_for_this_band.append(tup)
            #print(tup)
        if count==num_of_hash: 
            bands.append(hash_for_this_band)
            break
end = time.time()

print('time used to create the hash functions in all bands:\
', round((end-start),2), ' seconds')         
t1=end-start
#print(bands)


time used to create the hash functions in all bands: 0.0  seconds


calculating the results of all hash functions in bands on all 491 documents

In [89]:
start = time.time()
for k in range (0,num_of_bands):
    hash_results=[]
    for i in range(0,491):
        temp=[]
        for j in range (0,num_of_hash):
            temp.append(\
            minHash(matrix_text[i],bands[k][j][0],bands[k][j][1],34231))
        hash_results.append(temp)
    hash_results_bands.append(hash_results)  
end = time.time()

print('time used to compute hash results in bands \
on 491 texts: ', round((end-start),2), ' seconds') 
t2=end-start

time used to compute hash results in bands on 491 texts:  4.37  seconds


# Extract the pairs from the similar hash results in bands

In [85]:
start = time.time()

for i in range(0,num_of_bands):
    a= partition(hash_results_bands[i])
    pair_producer(a)
end = time.time()
print('It took', round((end-start),2), 'seconds \
to calculate pairs from minhash results\n bands=',num_of_bands, \
      'hash functions= ', num_of_hash) 
t3=end-start


It took 31.15 seconds to calculate pairs from minhash results
 bands= 10 hash functions=  2


# Check False Positives and False Negatives

In [86]:
pos=0
neg=0
for i in range (0,491):
    for j in range (i+1,491):
        if jac_sim_matrix[i][j]>=0.3:
            pos+=1
        else:
            neg+=1

print('real positives: ', pos)
print('real negatives: ', neg)
            

real positives:  47760
real negatives:  72535


In [87]:
false_positive=0
false_negative=0
checked_pairs_matrix=np.zeros((491, 491))

for i in range (0,len(hash_pairs)):
    checked_pairs_matrix[hash_pairs[i][0]][hash_pairs[i][1]]=-1
    if jac_sim_matrix[hash_pairs[i][0]][hash_pairs[i][1]]<0.3:
        false_positive+=1

        
for i in range (0,491):
    for j in range (i+1,491):
        if checked_pairs_matrix[i,j]==0:
            if jac_sim_matrix[i][j]>=0.3:
                false_negative+=1

print("#{False Positive}     ", false_positive)
print("َ#{All selected Pairs} ",len(hash_pairs))
print("#{False Negative}     ", false_negative)

#{False Positive}      19657
َ#{All selected Pairs}  45116
#{False Negative}      22301


In [88]:
tp= len(hash_pairs)-false_positive
prec= tp / (tp + false_positive)
rec= tp / (tp + false_negative)
f_one_score= 1/ ((1/rec)+(1/prec))
print("f1_score ", round(f_one_score,3))
print("time ", round (t1+t2+t3+14.4,2), 'seconds' )  #14.4 was time to calculate 
                                                    #universal set and shingles memberships

f1_score  0.274
time  49.96 seconds
